<a id='top'></a>
# How to slice and dice the data
Below are a series of examples on how to slice and dice the data that is stored in the *.sqlite* file generated by the [MorningStar.com](https://www.morningstar.com) web scraper.

##### NOTE:
- The data used in the code below come from the *.sqlite* file that is automatically generated by the web scraper once it has been installed and ran locally on your machine. See [README]() for instructions on how to run install and run the scraper.
- Navigation links only when using [Jupyter notebook](https://jupyter.org/).


**Content**

1. [Required modules and matplotlib backend](#modules)
1. [Creating a master (bridge table) DataFrame instance using the DataFrames class](#master)
1. [Methods for creating DataFrame instances](#methods)
    1. `quoteheader` - [MorningStar (MS) Quote Header](#quote)
    1. `valuation` - [MS Valuation table with Price Ratios (P/E, P/S, P/B, P/C) for the past 10 yrs](#val)
    1. `keyratios` - [MS Ratio - Key Financial Ratios & Values](#keyratios)
    1. `finhealth` - [MS Ratio - Financial Health](#finhealth)
    1. `profitability` - [MS Ratio - Profitability](#prof)
    1. `growth` - [MS Ratio - Growth](#growth)
    1. `cfhealth` - [MS Ratio - Cash Flow Health](#cfh)
    1. `efficiency` - [MS Ratio - Efficiency](#eff)
    1. `annualIS` - [MS Annual Income Statements](#isa)
    1. `quarterlyIS` - [MS Quarterly Income Statements](#isq)
    1. `annualBS` - [MS Annual Balance Sheets](#bsa)
    1. `quarterlyBS` - [MS Quarterly Balance Sheets](#bsq)
    1. `annualCF` - [MS Annual Cash Flow Statements](#cfa)
    1. `quarterlyCF` - [MS Quarterly Cash Flow Statements](#cfq)
    1. `insider_trades` - [Insider Transactions](#it)
1. [Performing statistical analysis](#stats)
    1. [Count of database records](#stats)
    1. [Last updated dates](#lastupdate)
    1. [Number of records by security type](#type)
    1. [Number of records by country, based on the location of exchanges](#country)
    1. [Number of records per exchange](#exchange)
    1. [Number of stocks by sector](#sector)
    1. [Number of stocks by industry](#industry)
    1. [Mean price ratios (P/E, P/S, P/B, P/CF) of stocks by sectors](#meanpr)
1. [Applying various criteria to filter common stocks](#value) *(in progress)*
1. [Additional sample / test code](#additional) *(in progress)*

<a id="modules"></a>
# Required modules and matplotlib backend

In [9]:
!pip3 install pandas
!pip3 install numpy
!pip3 install IPython
#!pip3 install sqlite3
#!pip3 install json
# !pip3 install sys
# !pip3 install re
# !pip3 install os

In [10]:
#!pip3 install matplotlib.pyplot
!pip3 install matplotlib
!pip3 install pandas
!pip3 install numpy
#!pip3 install dataframes #module containing class used to create DataFrame objects from SQLite database file
#!pip3 install datetime
#!pip3 install re
!pip3 install requests

In [11]:



class DataFrames():

    db_file = 'datab/mstables.sqlite' # Standard db file name

    def __init__(self, file = db_file):

        msg = 'Creating initial DataFrames objects from file {}...\n'
        print(msg.format(file))

        self.conn = sqlite3.connect(
            file, detect_types=sqlite3.PARSE_COLNAMES)
        self.cur = self.conn.cursor()

        # Row Headers
        colheaders = self.table('ColHeaders', True)
        self.colheaders = colheaders.set_index('id')

        # Dates and time references
        timerefs = self.table('TimeRefs', True)
        self.timerefs = timerefs.set_index('id').replace(['', '—'], None)

        # Reference tables
        self.urls = self.table('URLs', True)
        self.securitytypes = self.table('SecurityTypes', True)
        self.tickers = self.table('Tickers', True)
        self.sectors = self.table('Sectors', True)
        self.industries = self.table('Industries', True)
        self.styles = self.table('StockStyles', True)
        self.exchanges = self.table('Exchanges', True)
        self.countries = (self.table('Countries', True)
            .rename(columns={'a2_iso':'country_c2', 'a3_un':'country_c3'}))
        self.companies = self.table('Companies', True)
        self.currencies = self.table('Currencies', True)
        self.stocktypes = self.table('StockTypes', True)

        #self.fetchedurls = self.table('Fetched_urls', True)

        # Master table
        self.master0 = self.table('Master', True)

        # Merge Tables
        self.master = (self.master0
        # Ticker Symbols
         .merge(self.tickers, left_on='ticker_id', right_on='id')
         .drop(['id'], axis=1)
        # Company / Security Name
         .merge(self.companies, left_on='company_id', right_on='id')
         .drop(['id', 'company_id'], axis=1)
        # Exchanges
         .merge(self.exchanges, left_on='exchange_id', right_on='id')
         .drop(['id'], axis=1)
        # Industries
         .merge(self.industries, left_on='industry_id', right_on='id')
         .drop(['id', 'industry_id'], axis=1)
        # Sectors
         .merge(self.sectors, left_on='sector_id', right_on='id')
         .drop(['id', 'sector_id'], axis=1)
        # Countries
         .merge(self.countries, left_on='country_id', right_on='id')
         .drop(['id', 'country_id'], axis=1)
        # Security Types
         .merge(self.securitytypes, left_on='security_type_id', right_on='id')
         .drop(['id', 'security_type_id'], axis=1)
        # Stock Types
         .merge(self.stocktypes, left_on='stock_type_id', right_on='id')
         .drop(['id', 'stock_type_id'], axis=1)
        # Stock Style Types
         .merge(self.styles, left_on='style_id', right_on='id')
         .drop(['id', 'style_id'], axis=1)
        # Quote Header Info
         .merge(self.quoteheader(), on=['ticker_id', 'exchange_id'])
         .rename(columns={'fpe':'PE_Forward'})
        # Currency
         .merge(self.currencies, left_on='currency_id', right_on='id')
         .drop(['id', 'currency_id'], axis=1)
        # Fiscal Year End
         .merge(self.timerefs, left_on='fyend_id', right_on='id')
         .drop(['fyend_id'], axis=1)
         .rename(columns={'dates':'fy_end'})
        )
        # Change date columns to TimeFrames
        self.master['fy_end'] = pd.to_datetime(self.master['fy_end'])
        self.master['update_date'] = pd.to_datetime(self.master['update_date'])
        self.master['lastdate'] = pd.to_datetime(self.master['lastdate'])
        self.master['_52wk_hi'] = self.master['_52wk_hi'].astype('float')
        self.master['_52wk_lo'] = self.master['_52wk_lo'].astype('float')
        self.master['lastprice'] = self.master['lastprice'].astype('float')
        self.master['openprice'] = self.master['openprice'].astype('float')

        print('\nInitial DataFrames created successfully.')


    def quoteheader(self):
        return self.table('MSheader')


    def valuation(self):
        # Create DataFrame
        val = self.table('MSvaluation')

        # Rename column headers with actual year values
        yrs = val.iloc[0, 2:13].replace(self.timerefs['dates']).to_dict()
        cols = val.columns[:13].values.tolist() + list(map(
            lambda col: ''.join([col[:3], yrs[col[3:]]]), val.columns[13:]))
        val.columns = cols

        # Resize and reorder columns
        val = val.set_index(['exchange_id', 'ticker_id']).iloc[:, 11:]

        return val


    def keyratios(self):
        keyr = self.table('MSfinancials')
        yr_cols = ['Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6',
            'Y7', 'Y8', 'Y9', 'Y10']
        keyr = self.get_yrcolumns(keyr, yr_cols)
        keyr[yr_cols[:-1]] = keyr[yr_cols[:-1]].astype('datetime64')

        return keyr


    def finhealth(self):
        finan = self.table('MSratio_financial')
        yr_cols = [col for col in finan.columns if col.startswith('fh_Y')]
        finan = self.get_yrcolumns(finan, yr_cols)
        finan[yr_cols[:-1]] = finan[yr_cols[:-1]].astype('datetime64')

        return finan


    def profitability(self):
        profit= self.table('MSratio_profitability')
        yr_cols = [col for col in profit.columns if col.startswith('pr_Y')]
        profit = self.get_yrcolumns(profit, yr_cols)
        profit[yr_cols[:-1]] = profit[yr_cols[:-1]].astype('datetime64')

        return profit


    def growth(self):
        growth = self.table('MSratio_growth')
        yr_cols = [col for col in growth.columns if col.startswith('gr_Y')]
        growth = self.get_yrcolumns(growth, yr_cols)
        growth[yr_cols[:-1]] = growth[yr_cols[:-1]].astype('datetime64')

        return growth


    def cfhealth(self):
        cfhealth = self.table('MSratio_cashflow')
        yr_cols = [col for col in cfhealth.columns if col.startswith('cf_Y')]
        cfhealth = self.get_yrcolumns(cfhealth, yr_cols)
        cfhealth[yr_cols[:-1]] = cfhealth[yr_cols[:-1]].astype('datetime64')

        return cfhealth


    def efficiency(self):
        effic = self.table('MSratio_efficiency')
        yr_cols = [col for col in effic.columns if col.startswith('ef_Y')]
        effic = self.get_yrcolumns(effic, yr_cols)
        effic[yr_cols[:-1]] = effic[yr_cols[:-1]].astype('datetime64')

        return effic

    # Income Statement - Annual
    def annualIS(self):
        rep_is_yr = self.table('MSreport_is_yr')
        yr_cols = [col for col in rep_is_yr.columns
                    if col.startswith('Year_Y')]
        rep_is_yr = self.get_yrcolumns(rep_is_yr, yr_cols)
        rep_is_yr[yr_cols[:-1]] = rep_is_yr[yr_cols[:-1]].astype('datetime64')

        return rep_is_yr

    # Income Statement - Quarterly
    def quarterlyIS(self):
        rep_is_qt = self.table('MSreport_is_qt')
        yr_cols = [col for col in rep_is_qt.columns
                    if col.startswith('Year_Y')]
        rep_is_qt = self.get_yrcolumns(rep_is_qt, yr_cols)
        rep_is_qt[yr_cols[:-1]] = rep_is_qt[yr_cols[:-1]].astype('datetime64')

        return rep_is_qt

    # Balance Sheet - Annual
    def annualBS(self):
        rep_bs_yr = self.table('MSreport_bs_yr')
        yr_cols = [col for col in rep_bs_yr.columns
                    if col.startswith('Year_Y')]
        rep_bs_yr = self.get_yrcolumns(rep_bs_yr, yr_cols)
        rep_bs_yr[yr_cols[:-1]] = rep_bs_yr[yr_cols[:-1]].astype('datetime64')

        return rep_bs_yr

    # Balance Sheet - Quarterly
    def quarterlyBS(self):
        rep_bs_qt = self.table('MSreport_bs_qt')
        yr_cols = [col for col in rep_bs_qt.columns
                    if col.startswith('Year_Y')]
        rep_bs_qt = self.get_yrcolumns(rep_bs_qt, yr_cols)
        rep_bs_qt[yr_cols[:-1]] = rep_bs_qt[yr_cols[:-1]].astype('datetime64')

        return rep_bs_qt

    # Cashflow Statement - Annual
    def annualCF(self):
        rep_cf_yr = self.table('MSreport_cf_yr')
        yr_cols = [col for col in rep_cf_yr.columns
                    if col.startswith('Year_Y')]
        rep_cf_yr = self.get_yrcolumns(rep_cf_yr, yr_cols)
        rep_cf_yr[yr_cols[:-1]] = rep_cf_yr[yr_cols[:-1]].astype('datetime64')

        return rep_cf_yr

    # Cashflow Statement - Quarterly
    def quarterlyCF(self):
        rep_cf_qt = self.table('MSreport_cf_qt')
        yr_cols = [col for col in rep_cf_qt.columns
                    if col.startswith('Year_Y')]
        rep_cf_qt = self.get_yrcolumns(rep_cf_qt, yr_cols)
        rep_cf_qt[yr_cols[:-1]] = rep_cf_qt[yr_cols[:-1]].astype('datetime64')

        return rep_cf_qt

    # 10yr Price History
    def priceHistory(self):

        return self.table('MSpricehistory')


    def insider_trades(self):
        df_insiders = self.table('Insiders', False)
        df_tradetypes = self.table('TransactionType', False)
        df_trades = self.table('InsiderTransactions', False)
        df_trades['date'] = pd.to_datetime(df_trades['date'])
        df = (df_trades
            .merge(df_insiders, left_on='name_id', right_on='id')
            .drop(['id', 'name_id'], axis=1)
            .merge(df_tradetypes, left_on='transaction_id', right_on='id')
            .drop(['id', 'transaction_id'], axis=1)
            )
        return df


    def get_yrcolumns(self, df, cols):
        for yr in cols:
            df = (df.merge(self.timerefs, left_on=yr, right_on='id')
                .drop(yr, axis=1).rename(columns={'dates':yr}))

        return df


    def table(self, tbl, prnt = False):
        self.cur.execute('SELECT * FROM {}'.format(tbl))
        cols = list(zip(*self.cur.description))[0]

        try:
            if prnt == True:
                msg = '\t- DataFrame \'df.{}\' ...'
                print(msg.format(tbl.lower()))
            return pd.DataFrame(self.cur.fetchall(), columns=cols)
        except:
            raise


    def __del__(self):
        self.cur.close()
        self.conn.close()


In [12]:
# %matplotlib notebook

import matplotlib.pyplot as plt
from src import dataframes, fetch
import datetime as DT
import re
import sqlite3
# from mstables import parse


# Reload in case changes have been made to module file
from importlib import reload
reload(dataframes);

FileNotFoundError: [Errno 2] No such file or directory: '../input//api.json'

In [5]:
db_file_name = 'data/mstables.sqlite'

while True:
        try:
            conn = sqlite3.connect(db_file_name)
            cur = conn.cursor()
        except sqlite3.OperationalError as S:
            fetch.print_('')
            print('\tError - sqlite3 error: {}'.format(S))
            continue
        except KeyboardInterrupt:
            print('\nGoodbye!')
            exit()
        except:
            raise
        break

# Get list of fetched urls from Fetched_urls
"""
cols = 'url_id, ticker_id, exch_id, fetch_date, source_text'
sql = '''SELECT {} FROM Fetched_urls
        WHERE status_code = 200 AND source_text IS NOT NULL
        ORDER BY ticker_id asc, url_id desc'''
sql = sql.format(cols)
fetched = fetch.db_execute(cur, sql).fetchall()


# Call parsing methods
parsing(conn, cur, fetched)

"""
# Save db and close db connection
# fetch.save_db(conn)
cur.close()
conn.close()
fetched = None

NameError: name 'sqlite3' is not defined

[return to the top](#top)
<a id="master"></a>
# Creating the master DataFrame instance
The DataFrames class is part of the [dataframes module](src/dataframes.py)

In [28]:
db_file_name = 'mstables' # SQLite database file that contains the data to be analyzed
df = dataframes.DataFrames(f'db/{db_file_name}.sqlite')
df

Creating initial DataFrames objects from file db/mstables.sqlite...

	- DataFrame 'df.colheaders' ...
	- DataFrame 'df.timerefs' ...
	- DataFrame 'df.urls' ...
	- DataFrame 'df.securitytypes' ...
	- DataFrame 'df.tickers' ...
	- DataFrame 'df.sectors' ...
	- DataFrame 'df.industries' ...
	- DataFrame 'df.stockstyles' ...
	- DataFrame 'df.exchanges' ...
	- DataFrame 'df.countries' ...
	- DataFrame 'df.companies' ...
	- DataFrame 'df.currencies' ...
	- DataFrame 'df.stocktypes' ...
	- DataFrame 'df.master' ...

Initial DataFrames created successfully.


In [29]:
print(df.timerefs, '\nhere are timerefs\n',
df.urls,'\nhere are urls\n',
df.securitytypes,'\nhere are securitytypes\n',
df.tickers,'\nhere are tickers\n',
df.sectors,'\nhere are sectors\n',
df.industries,'\nhere are industries\n',
df.exchanges,'\nhere are exchanges\n',
df.countries,'\nhere are countries\n',
df.companies,'\nhere are companies\n',
df.currencies,'\nhere are currencies\n',
df.stocktypes,'\nhere are stocktype\n')


display(df)
#df.stockstyles has an issue?

             dates
id                
1             None
2       2012-06-30
3       2021-12-31
9       2004-09-30
10      2012-12-31
...            ...
87635   2018-12-30
89191   2017-07-31
96482   2022-04-19
98856   2009-09-30
102475  2012-02-28

[343 rows x 1 columns] 
here are timerefs
     id                                                url
0    1  https://www.morningstar.com/api/v2/search/Secu...
1    2  https://www.morningstar.com/api/v2/search/Secu...
2    3  https://www.morningstar.com/api/v2/search/secu...
3    4  http://quotes.morningstar.com/stockq/c-company...
4    5  http://quotes.morningstar.com/stockq/c-header?...
5    6  http://financials.morningstar.com/valuate/valu...
6    7  http://financials.morningstar.com/finan/financ...
7    8  http://financials.morningstar.com/finan/financ...
8    9  http://performance.morningstar.com/perform/Per...
9   10  http://financials.morningstar.com/ajax/ReportP...
10  11  http://financials.morningstar.com/ajax/ReportP...
11  12  http:

## Creating Master DataFrame instance from reference tables
Merge `df.master` (*Master* bridge table) with other reference tables (e.g. `df.tickers`, `df.exchanges`, etc.) and filter out inactive / invalid records.
### DataFrame Instance
Create `df_master`

In [8]:
import pandas as pd

# Assuming these DataFrames are already defined in your script
# df.urls, df.securitytypes, df.tickers, df.sectors, df.industries, df.stockstyles
# df.exchanges, df.countries, df.companies, df.currencies, df.stocktypes

# Perform sequential merges on a common key (assuming 'ticker_id' is the shared key)

# Step 1: Merge df.urls with df.securitytypes on 'ticker_id'
df_master = pd.merge(df.urls, df.securitytypes, on='id', how='left')

# Step 2: Merge df_master with df.tickers
df_master = pd.merge(df_master, df.tickers, on='ticker_id', how='left')

# Step 3: Merge with df.sectors
# df_master = pd.merge(df_master, df.sectors, on='ticker_id', how='left')

# Step 4: Merge with df.industries
df_master = pd.merge(df_master, df.industries, on='ticker_id', how='left')

# Step 5: Merge with df.stockstyles
# df_master = pd.merge(df_master, df.stockstyles, on='ticker_id', how='left')

# Step 6: Merge with df.exchanges
##df_master = pd.merge(df_master, df.exchanges, on='ticker_id', how='left')

# Step 7: Merge with df.countries
df_master = pd.merge(df_master, df.countries, on='ticker_id', how='left')

# Step 8: Merge with df.companies
df_master = pd.merge(df_master, df.companies, on='ticker_id', how='left')

# Step 9: Merge with df.currencies
## df_master = pd.merge(df_master, df.currencies, on='ticker_id', how='left')

# Step 10: Merge with df.stocktypes
df_master = pd.merge(df_master, df.stocktypes, on='ticker_id', how='left')

# Now df_master contains all the combined information
print(df_master.head())

# Optionally, save the merged DataFrame to a CSV file or a new SQLite database
df_master.to_csv('merged_data.csv', index=False)
# Or to SQLite
# df_master.to_sql('master_table', conn, if_exists='replace', index=False)


NameError: name 'df' is not defined

In [ ]:
df_master_copy = df.master.copy()
df_master_copy

In [ ]:
DT.date.today()


In [ ]:
DT.datetime(2024,month=10,day=9)

Apply the following filter:
<br>
- $lastdate < cutoff\_date$
<br>
- $cutoff\_date = date one\ week\ prior\ to\ last\ date\ the\ database\ was\ updated$
<br>
<br>
This filter ensure that only active records are included in the master dataframe, excluding inactive MorningStar records which are no longer being updated on a regular basis. These inactive records are typically symbols that are no longer active in their exchange.

cutoff_days = 10
df_updated_ct = df_master[['update_date', 'ticker']].groupby('update_date').count().sort_index()
cutoff_date = df_updated_ct[df_updated_ct['ticker'] > 100].index[0] # - DT.timedelta(days=cutoff_days)

df_master = df_master.where(df_master['lastdate'] >= cutoff_date).dropna(axis=0, how='all')

In [ ]:
cutoff_days = 1
df_updated_ct = df_master[['update_date', 'ticker']].groupby('update_date').count().sort_index()
cutoff_date = df_updated_ct[df_updated_ct['ticker'] > 100].index[0] # - DT.timedelta(days=cutoff_days)

df_master = df_master.where(df_master['lastdate'] >= cutoff_date).dropna(axis=0, how='all')

In [ ]:
df_master

#### DataFrame Size

In [ ]:
msg = 'DataFrame df_master contains {:,.0f} records and {:,.0f} columns.'
print(msg.format(df_master.shape[0], df_master.shape[1]))

#### DataFrame Columns

In [ ]:
df_master.columns.values

<br></br>
[return to the top](#top)
<a id='methods'></a>
# Creating DataFrame instances with dataframes methods
Class DataFrames from [dataframe.py](dataframe.py) contains the following methods that return a pd.DataFrame object for the specified database table:

- `quoteheader` - [MorningStar (MS) Quote Header](#quote)
- `valuation` - [MS Valuation table with Price Ratios (P/E, P/S, P/B, P/C) for the past 10 yrs](#val)
- `keyratios` - [MS Ratio - Key Financial Ratios & Values](#keyratios)
- `finhealth` - [MS Ratio - Financial Health](#finhealth)
- `profitability` - [MS Ratio - Profitability](#prof)
- `growth` - [MS Ratio - Growth](#growth)
- `cfhealth` - [MS Ratio - Cash Flow Health](#cfh)
- `efficiency` - [MS Ratio - Efficiency](#eff)
- `annualIS` - [MS Annual Income Statements](#isa)
- `quarterlyIS` - [MS Quarterly Income Statements](#isq)
- `annualBS` - [MS Annual Balance Sheets](#bsa)
- `quarterlyBS` - [MS Quarterly Balance Sheets](#bsq)
- `annualCF` - [MS Annual Cash Flow Statements](#cfa)
- `quarterlyCF` - [MS Quarterly Cash Flow Statements](#cfq)
- `insider_trades` - [Insider Transactions](#it)

<a id='quote'></a>
### Quote Header
##### DataFrame Instance

In [ ]:
df_quote = df.quoteheader()

In [ ]:
df_quote.head()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_quote)))

<a id='val'></a>
[return to the top](#top)
### Price Ratios (P/E, P/S, P/B, P/C)
##### DataFrame Instance

In [ ]:
df_vals = df.valuation().reset_index()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_vals)))

##### DataFrame Columns

In [ ]:
df_vals.columns

<a id='keyratios'></a>
[return to the top](#top)
### Key Ratios
##### DataFrame Instance

In [ ]:
df_keyratios = df.keyratios()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_keyratios)))

##### DataFrame Columns

In [ ]:
df_labels_keyratios = (df_keyratios
                     .loc[0, [col for col in df_keyratios.columns if 'Y' not in col and col.startswith('i')]]
                     .replace(df.colheaders['header']))
df_labels_keyratios

<a id='finhealth'></a>
[return to the top](#top)
### Financial Health
##### DataFrame Instance

In [ ]:
df_finhealth = df.finhealth()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_finhealth)))

##### DataFrame Columns

In [ ]:
df_labels_finhealth = (df_finhealth.loc[0, [col for col in df_finhealth.columns
                                          if 'Y' not in col and '_id' not in col]]
                     .replace(df.colheaders['header']))
df_labels_finhealth

<a id='prof'></a>
[return to the top](#top)
### Profitability
##### DataFrame Instance

In [ ]:
df_profitab = df.profitability()

In [ ]:
df_profitab.head()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_profitab)))

##### DataFrame Columns

In [ ]:
df_labels_profitab = (df_profitab.loc[0, [col for col in df_profitab.columns if 'Y' not in col and '_id' not in col]]
                    .replace(df.colheaders['header']))
df_labels_profitab

<a id='growth'></a>
[return to the top](#top)
### Growth
##### DataFrame Instance

In [ ]:
df_growth = df.growth()

In [ ]:
df_growth.head()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_growth)))

##### DataFrame Columns

In [ ]:
df_labels_growth = (df_growth.loc[0, [col for col in df_growth.columns
                                      if 'Y' not in col and '_id' not in col]].replace(df.colheaders['header']))
df_labels_growth

<a id='cfh'></a>
[return to the top](#top)
### Cash Flow Health
##### DataFrame Instance

In [ ]:
df_cfhealth = df.cfhealth()

In [ ]:
df_cfhealth.head()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_cfhealth)))

##### DataFrame Columns

In [ ]:
df_labels_cfhealth = df_cfhealth.loc[0, [col for col in df_cfhealth.columns if 'Y' not in col
                                         and '_id' not in col]].replace(df.colheaders['header'])
df_labels_cfhealth

<a id='eff'></a>
[return to the top](#top)
### Efficiency
##### DataFrame Instance

In [ ]:
df_efficiency = df.efficiency()

In [ ]:
df_efficiency.head()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_efficiency)))

##### DataFrame Columns

In [ ]:
# Financial Health DataFrame Columns
(df_efficiency.loc[0, [col for col in df_efficiency.columns if 'Y' not in col and '_id' not in col]]
 .replace(df.colheaders['header']))

<a id='isa'></a>
[return to the top](#top)
### Annual Income Statement
##### DataFrame Instance

In [ ]:
df_annualIS = df.annualIS()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_annualIS)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_annualIS if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_annualIS[label].unique().tolist() if pd.notna(header)]

df_labels_aIS = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_aIS['value'] = df_labels_aIS['value'].replace(df.colheaders['header'])
df_labels_aIS[df_labels_aIS['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_aIS.values.tolist(), df_labels_aIS.index)))

<a id='isq'></a>
[return to the top](#top)
### Quarterly Income Statements
##### DataFrame Instance

In [ ]:
df_quarterlyIS = df.quarterlyIS()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_quarterlyIS)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_annualIS if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_annualIS[label].unique().tolist() if pd.notna(header)]

df_labels_aIS = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_aIS['value'] = df_labels_aIS['value'].replace(df.colheaders['header'])
df_labels_aIS[df_labels_aIS['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_aIS.values.tolist(), df_labels_aIS.index)))

<a id='bsa'></a>
[return to the top](#top)
### Annual Balance Sheet
##### DataFrame Instance

In [ ]:
df_annualBS = df.annualBS()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_annualBS)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_annualBS if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_annualBS[label].unique().tolist() if pd.notna(header)]

df_labels_aBS = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_aBS['value'] = df_labels_aBS['value'].replace(df.colheaders['header'])
df_labels_aBS[df_labels_aBS['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_aBS.values.tolist(), df_labels_aBS.index)))

<a id='bsq'></a>
[return to the top](#top)
### Quarterly Balance Sheet
##### DataFrame Instance

In [ ]:
df_quarterlyBS = df.quarterlyBS()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_quarterlyBS)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_quarterlyBS if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_quarterlyBS[label].unique().tolist() if pd.notna(header)]

df_labels_qBS = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_qBS['value'] = df_labels_qBS['value'].replace(df.colheaders['header'])
df_labels_qBS[df_labels_qBS['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_qBS.values.tolist(), df_labels_qBS.index)))

<a id='cfa'></a>
[return to the top](#top)
### Annual Cash Flow Statement
##### DataFrame Instance

In [ ]:
df_annualCF = df.annualCF()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_annualCF)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_annualCF if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_annualCF[label].unique().tolist() if pd.notna(header)]

df_labels_aCF = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_aCF['value'] = df_labels_aCF['value'].replace(df.colheaders['header'])
df_labels_aCF[df_labels_aCF['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_aCF.values.tolist(), df_labels_aCF.index)))

<a id='cfq'></a>
[return to the top](#top)
### Quarterly Cash Flow Statement
##### DataFrame Instance

In [ ]:
df_quarterlyCF = df.quarterlyCF()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_quarterlyCF)))

##### DataFrame Columns

In [ ]:
labels = [col for col in df_quarterlyCF if 'label' in col]
labels = [[label, header] for label in labels
          for header in df_quarterlyCF[label].unique().tolist() if pd.notna(header)]

df_labels_qCF = (pd.DataFrame(labels, columns=['header', 'value']).set_index('header').astype('int'))
df_labels_qCF['value'] = df_labels_qCF['value'].replace(df.colheaders['header'])
df_labels_qCF[df_labels_qCF['value'].astype('str').str.contains('ncome')].sort_values(by='value')

sorted(list(zip(df_labels_qCF.values.tolist(), df_labels_qCF.index)))

<a id='it'></a>
[return to the top](#top)
### Insider Transactions
##### DataFrame Instance

In [ ]:
df_insidertrades = df.insider_trades()

##### DataFrame Length

In [ ]:
print('DataFrame contains {:,.0f} records.'.format(len(df_insidertrades)))

<a id="stats"></a>
[return to the top](#top)
# Performing statistical analysis
### Count of database records
**1.** Total number of records **before** merging reference tables (length of `df.master0`)

In [ ]:
print('DataFrame df.master contains {:,.0f} records.'.format(len(df.master0)))

**2.** Total number of records **after** merging reference tables (length of `df.master`)

In [ ]:
print('DataFrame df_master0 contains {:,.0f} records.'.format(len(df.master)))

**3.** Total number of records **after** filtering out inactive records (length of `df_master`)

In [ ]:
print('DataFrame df_master contains {:,.0f} records.'.format(len(df_master)))

<a id="lastupdate"></a>
[return to the top](#top)
### Last updated dates
List of dates (as a pd.Series object) of when the database records were last updated.
The values indicate the number of records updated on each date.

In [ ]:
(df_master[['update_date', 'ticker']].groupby(by='update_date').count().sort_index(ascending=False)
 .rename(columns={'ticker':'ticker_count'}))

<a id="type"></a>
[return to the top](#top)
### Number of records by Security Type

In [ ]:
(df_master[['security_type', 'ticker']].groupby(by='security_type').count()
 .rename(columns={'ticker':'ticker_count'}))

<a id="country"></a>
[return to the top](#top)
### Number of records by Country, based on the location of exchanges

In [ ]:
(df_master[['country', 'country_c3', 'ticker']]
 .groupby(by=['country', 'country_c3']).count().rename(columns={'ticker':'ticker_count'})
)

<a id="exchange"></a>
[return to the top](#top)
### Number of records per exchange
Where $ticker\_count > 100$

In [ ]:
cols = ['country', 'country_c3', 'exchange', 'exchange_sym', 'ticker']
df_exchanges = df_master[cols].groupby(by=cols[:-1]).count().rename(columns={'ticker':'ticker_count'})
df_exchanges[df_exchanges['ticker_count'] > 100].sort_values(by='ticker_count', ascending=False)


[return to the top](#top)
### Number of Stocks by Country of Exchange

In [ ]:
(df_master
 .where(df_master['security_type'] == 'Stock').dropna(axis=0, how='all')[['country', 'country_c3', 'ticker']]
 .groupby(by=['country', 'country_c3']).count().rename(columns={'ticker':'ticker_count'})
 .sort_values(by='ticker_count', ascending=False))

<a id="sector"></a>
[return to the top](#top)
### Number of stocks by sector

In [ ]:
(df_master
 .where((df_master['security_type'] == 'Stock') & (df_master['sector'] != '—')).dropna(axis=0, how='all')
 .groupby(by='sector').count()
 .rename(columns={'ticker':'stock_count'}))['stock_count'].sort_values(ascending=False)

<a id="industry"></a>
[return to the top](#top)
### Number of stocks by industry

In [ ]:
(df_master[['sector', 'industry', 'ticker']]
 .where((df_master['security_type'] == 'Stock') & (df_master['industry'] != '—')).dropna(axis=0, how='all')
 .groupby(by=['sector', 'industry']).count().rename(columns={'ticker':'stock_count'}))

<a id="meanpr"></a>
[return to the top](#top)
### Mean price ratios (P/E, P/S, P/B, P/CF) of stocks by sectors

First, merge `df_master` and `df_vals` and remove outliers where Price Ratio > 10,000

In [ ]:
df_valuation = (df_master
                .where((df_master['security_type'] == 'Stock') & (df_master['sector'] != '—'))
                .dropna(axis=0, how='all')
                .merge(df_vals, on=['ticker_id', 'exchange_id'])
                .drop(['ticker_id', 'exchange_id'], axis=1))

cols = list(filter(lambda col: col.startswith('P'), df_valuation))
df0 = df_valuation.copy()
for col in cols:
    df0 = df[(df[col] < 10000) | df[col].isna()]
print('There are {:,.0f} Stock records that fit this criteria.'.format(len(df)))

#### Mean TTM Price Ratios for all stocks:

In [ ]:
df_val_mean = (df[['sector', 'company']].groupby('sector').count()
               .rename(columns={'company':'count'})
               .merge(df.groupby('sector').mean().round(4), on='sector')
               .sort_values(by='PE_TTM', ascending=False))

df_val_mean[['count', 'PE_Forward', 'PE_TTM', 'PB_TTM', 'PS_TTM', 'PC_TTM']]

#### Mean TTM Price Ratios for USA stocks:

In [ ]:
df_valuation_USA = df[df['country_c3'] == 'USA']
print('There are {:,.0f} Stock records that fit this criteria.'.format(len(df_valuation_USA)))

df_val_mean_USA = (df_valuation_USA[['sector', 'company']].groupby('sector').count()
                   .rename(columns={'company':'count'})
                   .merge(df_valuation_USA.groupby('sector').mean().round(4), on='sector')
                   .sort_values(by='PE_TTM', ascending=False))

df_val_mean_USA[['count', 'PE_Forward', 'PE_TTM', 'PB_TTM', 'PS_TTM', 'PC_TTM']]

#### Mean TTM Price Ratios for DEU (Germany) stocks:

In [ ]:
df_valuation_DEU = df[df['country_c3'] == 'DEU']
print('There are {:,.0f} Stock records that fit this criteria.'.format(len(df_valuation_DEU)))

df_val_mean_DEU = (df_valuation_DEU[['sector', 'company']].groupby('sector').count()
                   .rename(columns={'company':'count'})
                   .merge(df_valuation_DEU.groupby('sector').mean().round(4), on='sector')
                   .sort_values(by='PE_TTM', ascending=False))

df_val_mean_DEU[['count', 'PE_Forward', 'PE_TTM', 'PB_TTM', 'PS_TTM', 'PC_TTM']]

#### Mean TTM Price Ratios for S&P 500 stocks:

In [ ]:
url = r'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
df_sp500 = tables[0]
df_sp500.columns = df_sp500.iloc[0]
df_sp500 = df_sp500.drop(0, axis=0).set_index('Symbol').join(df.set_index('ticker'))
df_sp500 = df_sp500[df_sp500['country_c3'] == 'USA'].fillna(0)

In [ ]:
print('There are {:,.0f} Stock records that fit this criteria.'.format(len(df_sp500)))

df_val_mean_sp500 = (df_sp500[['sector', 'company']].groupby('sector').count()
                     .rename(columns={'company':'count'})
                     .merge(df_sp500.groupby('sector').mean().round(4), on='sector')
                     .sort_values(by='PE_TTM', ascending=False))

df_val_mean_sp500[['count', 'PE_Forward', 'PE_TTM', 'PB_TTM', 'PS_TTM', 'PC_TTM']]

[return to the top](#top)
#### Plot of TTM P/E by Sectors
*All Stocks*

In [ ]:
fig_pe, (ax_pe, ax_pe_usa, ax_pe_deu) = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8, 6))

# All Stocks
pe = df_val_mean['PE_TTM']
x = [x*3 for x in range(len(pe))]
y = pe
bars = ax_pe.bar(x, y, width=2)
for bar in bars:
    ax_pe.text(bar.get_x()+1, bar.get_height()+1.5, '{:.1f}'.format(bar.get_height()),
               color='black', ha='center', fontsize=9)
ax_pe.get_children()[22].set_color(None)
ax_pe.get_children()[23].set_color(None)
ax_pe.get_children()[25].set_color(None)
ax_pe.set_title('All Stocks', loc='left', fontsize=9, fontweight='bold')

# USA
pe_usa = df_val_mean_USA['PE_TTM']
x = [x*3 for x in range(len(pe_usa))]
y = pe_usa
bars = ax_pe_usa.bar(x, y, width=2)
for bar in bars:
    ax_pe_usa.text(bar.get_x()+1, bar.get_height()+1.5, '{:.1f}'.format(bar.get_height()),
                   color='black', ha='center', fontsize=9)
ax_pe_usa.get_children()[22].set_color(None)
ax_pe_usa.get_children()[23].set_color(None)
ax_pe_usa.get_children()[25].set_color(None)
ax_pe_usa.set_title('USA', loc='left', fontsize=9, fontweight='bold')

# DEU
pe_deu = df_val_mean_DEU['PE_TTM']
x = [x*3 for x in range(len(pe_deu))]
y = pe_deu
bars = ax_pe_deu.bar(x, y, width=2)
for bar in bars:
    ax_pe_deu.text(bar.get_x()+1, bar.get_height()+1.5, '{:.1f}'.format(bar.get_height()),
                   color='black', ha='center', fontsize=9)
ax_pe_deu.get_children()[22].set_color(None)
ax_pe_deu.get_children()[23].set_color(None)
ax_pe_deu.get_children()[25].set_color(None)
ax_pe_deu.set_title('DEU', loc='left', fontsize=9, fontweight='bold')

# Plot adjustments
plt.xticks(ticks=x, labels=y.index.tolist(), fontsize=9)
plt.axis([-3, len(x)*3, 0, 100])
plt.suptitle('Average TTM P/E of Stocks by Sector for key regions', fontsize=11, fontweight='bold')
plt.yticks([])
plt.subplots_adjust(bottom=0.3, hspace=1)

for tick in ax_pe_deu.xaxis.get_ticklabels():
    tick.set_rotation(90)

[return to the top](#top)
#### P/E by Sector for past 10 yrs for US Stocks

In [ ]:
cols = list(filter(lambda col: col.startswith('PE_'), df.columns))[1:]

fig, axs = plt.subplots(11, 1, figsize=(7, 6), sharex=True)
sectors = df_val_mean_USA.index.values
x = list(range(11))

for sector, ax in zip(sectors, axs):
    y = df_val_mean_USA[cols].loc[sector].values
    p0 = ax.bar(x, y, width=0.5)
    #ax.set_title(sector, loc='left', fontsize=9, fontweight='bold')
    ax.spines['left'].set_color(None)
    ax.spines['right'].set_color(None)
    ax.spines['top'].set_color(None)
    for pt in list(zip(x, y)):
        if pt[1] > 0:
            ax.text(pt[0], pt[1] + max(y)*0.05, '{:.0f}'.format(pt[1]), ha='center', fontsize=8)
    ax.set_yticks([])
    ax.axis([-5.5, 11, 0, max(y)+10])
    ax.set_xlim(-5.5, 11)
    ax.text(-5.5, max(y)+10, sector, fontweight='bold', fontsize=8)
    ax0 = ax

plt.suptitle('US Stocks P/E by Sector for past 10 yrs', fontweight='bold', fontsize=11)
plt.subplots_adjust(top=0.91, bottom=0.08, hspace=0.4)
_ = plt.xticks(ticks=x, labels=list(map(lambda col: col[3:], cols)), fontsize=8, fontweight='bold')
for label in ax.xaxis.get_ticklabels():
    label.set_rotation(45)

[return to the top](#top)
#### P/S by Sector for past 10 yrs for US Stocks

In [ ]:
cols = list(filter(lambda col: col.startswith('PS_'), df.columns))

fig, axs = plt.subplots(11, 1, figsize=(7, 6), sharex=True)
sectors = df_val_mean_USA.index.values
x = list(range(11))

for sector, ax in zip(sectors, axs):
    y = df_val_mean_USA[cols].loc[sector].values
    p0 = ax.bar(x, y, width=0.5)
    #ax.set_title(sector, loc='left', fontsize=9, fontweight='bold')
    ax.spines['left'].set_color(None)
    ax.spines['right'].set_color(None)
    ax.spines['top'].set_color(None)
    for pt in list(zip(x, y)):
        if pt[1] > 0:
            ax.text(pt[0], pt[1] + max(y)*0.05, '{:.0f}'.format(pt[1]), ha='center', fontsize=8)
    ax.set_yticks([])
    ax.axis([-5.5, 11, 0, max(y)+10])
    ax.set_xlim(-5.5, 11)
    ax.text(-5.5, max(y)+10, sector, fontweight='bold', fontsize=8)
    ax0 = ax

plt.suptitle('US Stocks P/S by Sector for past 10 yrs', fontweight='bold', fontsize=11)
plt.subplots_adjust(top=0.91, bottom=0.08, hspace=0.4)
_ = plt.xticks(ticks=x, labels=list(map(lambda col: col[3:], cols)), fontsize=8, fontweight='bold')
for label in ax.xaxis.get_ticklabels():
    label.set_rotation(45)

[return to the top](#top)
### Stocks in the Cannabis Industry
Using stocks listed on [marijuanaindex.com](http://marijuanaindex.com/stock-quotes/north-american-marijuana-index/) under North America

In [ ]:
import json

with open('input/pot_stocks.json') as file:
    pot_symbols = json.loads(file.read())

pot_stocks = (pd.DataFrame(pot_symbols, columns=['ticker', 'country_c3'])
               .merge(df_master, how='left', on=['ticker', 'country_c3']).drop('country', axis=1)
               .rename(columns={'country_c3':'country', 'exchange_sym':'exch'}))

pot_stocks = (pot_stocks.where(((pot_stocks['country'] == 'USA') |
                                (pot_stocks['country'] == 'CAN')) &
                               (pot_stocks['sector'] != '—'))
              .dropna(axis=0, how='all').sort_values(by='company'))

In [ ]:
msg = 'Below are the {} stocks listed on marijuanaindex.com for North America.'
print(msg.format(len(pot_stocks['company'].unique())))

pot_stocks[['country', 'ticker', 'exch', 'company', 'sector', 'industry']]

<a id="value"></a>
[return to the top](#top)

# Applying various criteria to filter common stocks

Below is a list of different rules that can be applied to the data to screen stocks (development of *italicized rules* is still in progress)

- **[Rule 0](#rule99): CAGR > 7% for past 7 years**
- **[Rule 1](#rule1): No earnings deficit (loss) for past 5 or 7 years**
- **[Rule 2](#rule2): Uniterrupted and increasing Dividends for past 5 yrs**
- **[Rule 3](#rule3): P/E Ratio of 25 or less for the past 7 yrs and less then 20 for TTM**
- **[Rule 4](#rule4): Growth for the past year**
- **[Rule 5](#rule5): Current Ratio > 1.2**
- **[Rule 6](#rule6): Debt/Equity < 1.0**
- **[Rule 7](#rule7): Return on Equity > 10%**
- **[Rule X](#rulex): Stocks with insider buys in the past 3 months**

[Merge DataFrames](#mergerules) to screen stocks

[return to the top](#top)
<a id="rule99"></a>
## Rule 0. CAGR > 7% for past 5 years

Column labels in `df_keyratios`:

In [ ]:
df_labels_keyratios

### Revenue CAGR:

In [ ]:
iid = 'i0'
field = 'Rev'
label = 'CAGR_{}'.format(field)

df_rule0_Rev = (df_keyratios.where(df_keyratios['Y9'] > pd.to_datetime('2018-04-01')).dropna(axis=0, how='all'))

df_rule0_Rev[label] = 100 * ((df_rule0_Rev['{}_Y9'.format(iid)] / df_rule0_Rev['{}_Y4'.format(iid)]) ** (1/5) - 1)

cols = ['ticker_id', 'exchange_id', label] #, '{}_Y4'.format(iid), '{}_Y9'.format(iid)]
df_rule0_Rev = (df_rule0_Rev.where(df_rule0_Rev[label] >= 7).dropna(axis=0, how='all')
                .sort_values(by=label, ascending=False))[cols]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule0_Rev)))

### Operating Income CAGR:

In [ ]:
iid = 'i2'
field = 'OpeInc'
label = 'CAGR_{}'.format(field)

df_rule0_OpeInc = (df_keyratios.where(df_keyratios['Y9'] > pd.to_datetime('2018-04-01')).dropna(axis=0, how='all'))

df_rule0_OpeInc[label] = 100 * (
    (df_rule0_OpeInc['{}_Y9'.format(iid)] / df_rule0_OpeInc['{}_Y4'.format(iid)]) ** (1/5) - 1)

cols = ['ticker_id', 'exchange_id', label]
df_rule0_OpeInc = (df_rule0_OpeInc.where(df_rule0_OpeInc[label] >= 7).dropna(axis=0, how='all')
                .sort_values(by=label, ascending=False))[cols]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule0_OpeInc)))

### Operating Cash Flow CAGR:

In [ ]:
iid = 'i9'
field = 'OpeCF'
label = 'CAGR_{}'.format(field)

df_rule0_OpeCF = (df_keyratios.where(df_keyratios['Y9'] > pd.to_datetime('2018-04-01')).dropna(axis=0, how='all'))

df_rule0_OpeCF[label] = 100 * (
    (df_rule0_OpeCF['{}_Y9'.format(iid)] / df_rule0_OpeCF['{}_Y4'.format(iid)]) ** (1/5) - 1)

cols = ['ticker_id', 'exchange_id', label]
df_rule0_OpeCF = (df_rule0_OpeCF.where(df_rule0_OpeCF[label] >= 7).dropna(axis=0, how='all')
                .sort_values(by=label, ascending=False))[cols]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule0_OpeCF)))

### Free Cash Flow CAGR:

In [ ]:
iid = 'i11'
field = 'FreeCF'
label = 'CAGR_{}'.format(field)

df_rule0_FreeCF = (df_keyratios.where(df_keyratios['Y9'] > pd.to_datetime('2018-04-01')).dropna(axis=0, how='all'))
df_rule0_FreeCF[label] = 100 * (
    (df_rule0_FreeCF['{}_Y9'.format(iid)] / df_rule0_FreeCF['{}_Y4'.format(iid)]) ** (1/5) - 1)

cols = ['ticker_id', 'exchange_id', label]
df_rule0_FreeCF = (df_rule0_FreeCF.where(df_rule0_FreeCF[label] >= 7).dropna(axis=0, how='all')
                .sort_values(by=label, ascending=False))[cols]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule0_FreeCF)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule1"></a>
### Rule 1. No earnings deficit (loss) for past 5 or 7 years
Criteria: *"Find companies with positive earnings per share growth during the past five years with no earnings deficits. Earnings need to be higher in the most recent year than five years ago. Avoiding companies with earnings deficits during the past five years will help you stay clear of high-risk companies."* [(Source)](https://cabotwealth.com/daily/value-investing/benjamin-grahams-value-stock-criteria/)

#### 5 Years: (PENDING CORRECTION OF CODE)
*a. Identify Net Income column labels in* `df_annualIS`

In [ ]:
ilabel = 'Net income'
df_labels = df_labels_aIS[df_labels_aIS['value'] == ilabel].sort_values(by='value')
df_labels

*b. Get column headers for 'Net income' values for the past 5 yrs*

In [ ]:
# i_ids = [(label[-3:] + '_') for label in df_labels.index]

# def get_icols(col):
#     for i_id in i_ids:
#         if i_id in col:
#             return True
#     return False

# main_cols = ['ticker_id', 'exchange_id', 'country', 'exchange_sym', 'ticker', 'company', 'sector', 'industry',
#              'stock_type', 'style', 'Year_Y_6', 'Year_Y_5', 'Year_Y_4', 'Year_Y_3', 'Year_Y_2', 'Year_Y_1']
# data_cols = sorted(list(filter(get_icols, df_annualIS.columns)), key=lambda r: (r[-1], r[5:8]), reverse=True)

*c. Create 'Net Income' DataFrame*

In [ ]:
# df_annualIS1 = df_master.merge(df_annualIS, on=['ticker_id', 'exchange_id'])

# df_netinc5 = (df_annualIS1
#               .where((df_annualIS1['security_type'] == 'Stock') &
#                      (df_annualIS1['Year_Y_5'] > pd.to_datetime('2018-01')))
#               .dropna(axis=0, how='all')
#               .drop(['country'], axis=1)
#               .rename(columns={'country_c3':'country'})
#              )[main_cols + data_cols]

# np_netinc = df_netinc5[data_cols].values
# netinc_cols = [('NetIncome_Y' + data_cols[i * 3][-1], (i * 3, i * 3 + 1, i * 3 + 2))
#                for i in range(int(len(data_cols)/3))]

# vals = []
# for row in np_netinc:
#     row_vals = []
#     for i in range(len(netinc_cols)):
#         val = None
#         for col in netinc_cols[i][1]:
#             if not np.isnan(row[col]):
#                 val = row[col]
#                 break
#         row_vals.append(val)
#     vals.append(row_vals)

# df_netinc_vals = pd.DataFrame(vals, columns=list(zip(*netinc_cols))[0])
# df_netinc5 = df_netinc5[main_cols].join(df_netinc_vals)

In [ ]:
# df_rule1_5 = df_netinc5.where((df_netinc5['NetIncome_Y6'] > 0) &
#                             ((df_netinc5['NetIncome_Y5'] > 0) | (df_netinc5['NetIncome_Y5'].isna() & df_netinc5['NetIncome_Y4'].isna() & df_netinc5['NetIncome_Y3'].isna() & df_netinc5['NetIncome_Y2'].isna() & df_netinc5['NetIncome_Y1'].isna())) &
#                             ((df_netinc5['NetIncome_Y4'] > 0) | (df_netinc5['NetIncome_Y4'].isna() & df_netinc5['NetIncome_Y3'].isna() & df_netinc5['NetIncome_Y2'].isna() & df_netinc5['NetIncome_Y1'].isna())) &
#                             ((df_netinc5['NetIncome_Y3'] > 0) | (df_netinc5['NetIncome_Y3'].isna() & df_netinc5['NetIncome_Y2'].isna() & df_netinc5['NetIncome_Y1'].isna())) &
#                             ((df_netinc5['NetIncome_Y2'] > 0) | (df_netinc5['NetIncome_Y2'].isna() & df_netinc5['NetIncome_Y1'].isna())) &
#                             ((df_netinc5['NetIncome_Y1'] > 0) | (df_netinc5['NetIncome_Y1'].isna()))
#                            ).dropna(axis=0, how='all')

# df_rule1_5 = df_rule1_5[['ticker_id', 'exchange_id'] + df_rule1_5.columns.values.tolist()[-12:]]
# df_rule1_5.columns = [re.sub('Year_Y_', 'r1_Y', col) for col in df_rule1_5.columns]

# print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule1_5)))

#### 5 Years:

In [ ]:
cols = ['ticker_id', 'exchange_id'] + \
        [col for col in df_keyratios.columns if col.startswith('i4_') or col.startswith('Y')]

df_rule1_5 = (df_keyratios
              .where((df_keyratios['Y9'] >= pd.to_datetime('2018-04-01')) &
                     (df_keyratios['i4_Y10'] > 0) &
                     ((df_keyratios['i4_Y9'] > 0) | (df_keyratios['i4_Y9'].isna() & df_keyratios['i4_Y8'].isna() & df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna())) &
                     ((df_keyratios['i4_Y8'] > 0) | (df_keyratios['i4_Y8'].isna() & df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna())) &
                     ((df_keyratios['i4_Y7'] > 0) | (df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna())) &
                     ((df_keyratios['i4_Y6'] > 0) | (df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna())) &
                     ((df_keyratios['i4_Y5'] > 0) | (df_keyratios['i4_Y5'].isna())))
              .dropna(axis=0, how='all'))[cols]

df_rule1_5.columns = [re.sub('i4_', 'NetIncome_', col) for col in df_rule1_5.columns]
df_rule1_5.columns = [re.sub('^Y', 'r1_Y', col) for col in df_rule1_5.columns]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule1_5)))

#### 7 Years:

In [ ]:
cols = ['ticker_id', 'exchange_id'] + \
        [col for col in df_keyratios.columns if col.startswith('i4_') or col.startswith('Y')]

df_rule1_7 = (df_keyratios
              .where((df_keyratios['Y9'] >= pd.to_datetime('2018-04-01')) &
                     (df_keyratios['i4_Y10'] > 0) &
                     ((df_keyratios['i4_Y9'] > 0) | (df_keyratios['i4_Y9'].isna() & df_keyratios['i4_Y8'].isna() & df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna() & df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y8'] > 0) | (df_keyratios['i4_Y8'].isna() & df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna() & df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y7'] > 0) | (df_keyratios['i4_Y7'].isna() & df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna() & df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y6'] > 0) | (df_keyratios['i4_Y6'].isna() & df_keyratios['i4_Y5'].isna() & df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y5'] > 0) | (df_keyratios['i4_Y5'].isna() & df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y4'] > 0) | (df_keyratios['i4_Y4'].isna() & df_keyratios['i4_Y3'].isna())) &
                     ((df_keyratios['i4_Y3'] > 0) | (df_keyratios['i4_Y3'].isna())))
              .dropna(axis=0, how='all'))[cols]

df_rule1_7.columns = [re.sub('i4_', 'NetIncome_', col) for col in df_rule1_7.columns]
df_rule1_7.columns = [re.sub('^Y', 'r1_Y', col) for col in df_rule1_7.columns]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule1_7)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule2"></a>
### Rule 2. Uniterrupted and increasing *Dividends* for past 7 yrs

In [ ]:
df_labels_keyratios

In [ ]:
icol = df_labels_keyratios[df_labels_keyratios.str.contains('Dividends')].index[0]
icol

In [ ]:
main_cols = ['ticker_id', 'exchange_id',
             #'country_c3', 'exchange_sym', 'ticker', 'company',
             #'sector', 'industry', 'stock_type', 'style',
             'Y10', 'Y9', 'Y8', 'Y7', 'Y6', 'Y5']
icols = sorted([col for col in df_keyratios.columns if icol + '_' in col],
               key=lambda col: int(col[4:]), reverse=True)[:8]
icols

In [ ]:
df_rule2 = (df_keyratios
            .where((df_keyratios['Y9'] > pd.to_datetime('2018-04-01')) &
                   (df_keyratios['i6_Y10'] >= df_keyratios['i6_Y9']) &
                   ((df_keyratios['i6_Y9'] >= df_keyratios['i6_Y8']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna() & df_keyratios['i6_Y4'].isna() & df_keyratios['i6_Y5'].isna() & df_keyratios['i6_Y6'].isna() & df_keyratios['i6_Y7'].isna() & df_keyratios['i6_Y8'].isna())) &
                   ((df_keyratios['i6_Y8'] >= df_keyratios['i6_Y7']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna() & df_keyratios['i6_Y4'].isna() & df_keyratios['i6_Y5'].isna() & df_keyratios['i6_Y6'].isna() & df_keyratios['i6_Y7'].isna())) &
                   ((df_keyratios['i6_Y7'] >= df_keyratios['i6_Y6']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna() & df_keyratios['i6_Y4'].isna() & df_keyratios['i6_Y5'].isna() & df_keyratios['i6_Y6'].isna())) &
                   ((df_keyratios['i6_Y6'] >= df_keyratios['i6_Y5']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna() & df_keyratios['i6_Y4'].isna() & df_keyratios['i6_Y5'].isna())) &
                   ((df_keyratios['i6_Y5'] >= df_keyratios['i6_Y4']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna() & df_keyratios['i6_Y4'].isna())) &
                   ((df_keyratios['i6_Y4'] >= df_keyratios['i6_Y3']) | (df_keyratios['i6_Y2'].isna() & df_keyratios['i6_Y3'].isna())) &
                   ((df_keyratios['i6_Y3'] >= df_keyratios['i6_Y2']) | (df_keyratios['i6_Y2'].isna())))
            .dropna(axis=0, how='all').sort_values(by='Y9', ascending=False))[main_cols + icols]

df_rule2.columns = main_cols + [col.replace('i6', 'Dividend') for col in icols]
df_rule2.columns = [re.sub('^Y', 'r2_Y', col) for col in df_rule2.columns]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule2)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule3"></a>
### Rule 3. P/E Ratio of 25 or less for the past 7 yrs and less then 20 for TTM

In [ ]:
pe_cols = [col for col in df_vals.columns if 'PE_' in col]
pe_cols = ['ticker_id', 'exchange_id'] + [pe_cols[len(pe_cols)-i-1] for i in range(len(pe_cols))][:8]
pe_cols

In [ ]:
df_rule3 = (df_vals[pe_cols]
            .where((df_vals['PE_TTM'] <= 10) &
                   (df_vals['PE_2018'] <= 25) &
                   ((df_vals['PE_2017'] <= 25) | (df_vals['PE_2012'].isna() & df_vals['PE_2013'].isna() & df_vals['PE_2014'].isna() & df_vals['PE_2015'].isna() & df_vals['PE_2016'].isna() & df_vals['PE_2017'].isna())) &
                   ((df_vals['PE_2016'] <= 25) | (df_vals['PE_2012'].isna() & df_vals['PE_2013'].isna() & df_vals['PE_2014'].isna() & df_vals['PE_2015'].isna() & df_vals['PE_2016'].isna())) &
                   ((df_vals['PE_2015'] <= 25) | (df_vals['PE_2012'].isna() & df_vals['PE_2013'].isna() & df_vals['PE_2014'].isna() & df_vals['PE_2015'].isna())) &
                   ((df_vals['PE_2014'] <= 25) | (df_vals['PE_2012'].isna() & df_vals['PE_2013'].isna() & df_vals['PE_2014'].isna())) &
                   ((df_vals['PE_2013'] <= 25) | (df_vals['PE_2012'].isna() & df_vals['PE_2013'].isna())) &
                   ((df_vals['PE_2012'] <= 25) | (df_vals['PE_2012'].isna())))
            .dropna(axis=0, how='all').sort_values(by='PE_TTM'))

print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule3)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule4"></a>
## Rule 4. Growth for the past year

In [ ]:
df_labels_growth

### Revenue

In [ ]:
iid = 'i28'; label = '{}_gr_Y9'.format(iid); col = 'Rev';
cols = ['ticker_id', 'exchange_id', 'gr_Y9', label]
df_rule4_Rev = (df_growth[cols]
                .where((df_growth[label] > 0) & (df_growth['gr_Y9'] > pd.to_datetime('2018-04-01')))
                .dropna(axis=0, how='all').sort_values(by='gr_Y9')
                .rename(columns={label:'{}_Growth_Y9'.format(col)}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule4_Rev)))

### Operating Income

In [ ]:
iid = 'i32'; label = '{}_gr_Y9'.format(iid); col = 'OpeInc';
cols = ['ticker_id', 'exchange_id', 'gr_Y9', label]
df_rule4_OpeInc = (df_growth[cols]
                .where((df_growth[label] > 0) & (df_growth['gr_Y9'] > pd.to_datetime('2018-04-01')))
                .dropna(axis=0, how='all').sort_values(by='gr_Y9')
                .rename(columns={label:'{}_Growth_Y9'.format(col)}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule4_OpeInc)))

### Net Income

In [ ]:
iid = 'i81'; label = '{}_gr_Y9'.format(iid); col = 'NetInc';
cols = ['ticker_id', 'exchange_id', 'gr_Y9', label]
df_rule4_NetInc = (df_growth[cols]
                .where((df_growth[label] > 0) & (df_growth['gr_Y9'] > pd.to_datetime('2018-04-01')))
                .dropna(axis=0, how='all').sort_values(by='gr_Y9')
                .rename(columns={label:'{}_Growth_Y9'.format(col)}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule4_NetInc)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule5"></a>
### Rule 5. Current Ratio > 1.2

In [ ]:
df_labels_finhealth[-5:]

In [ ]:
col = 'i65_lfh_Y10'
df_rule5 = (df_finhealth[['ticker_id', 'exchange_id', col]]
            .where((df_finhealth[col] > 1.2) | (df_finhealth[col].isna()))
            .dropna(axis=0, how='all')
            .rename(columns={col:'current_ratio'}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule5)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule6"></a>
### Rule 6. Debt/Equity < 1.5

In [ ]:
col = 'i68_lfh_Y10'
df_rule6 = (df_finhealth[['ticker_id', 'exchange_id', col]]
            .where((df_finhealth[col] < 1.5) | (df_finhealth[col].isna()))
            .dropna(axis=0, how='all')
            .rename(columns={col:'debt2equity'}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule6)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule7"></a>
### Rule 7. Return on Equity > 10%

In [ ]:
df_labels_profitab[-9:]

In [ ]:
col = 'i26_pr_pro_Y10'
df_rule7 = (df_profitab[['ticker_id', 'exchange_id', col]]
            .where((df_profitab[col] > 10) | (df_profitab[col].isna()))
            .dropna(axis=0, how='all')
            .rename(columns={col:'return_on_equity'}))
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule7)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rule7"></a>
### Rule 8. P/B < 1.0

In [ ]:
df_rule8 = df_vals.where(df_vals['PB_TTM'] <= 1).dropna(axis=0, how='all')
print('Total of {:,.0f} records meet this criterium.'.format(len(df_rule8)))

[return to top of this section](#value),
[return to the top](#top)
<a id="rulex"></a>
### Rule X. Stocks with insider buys in the past 3 months

In [ ]:
datefilter = pd.to_datetime(DT.date.today()-DT.timedelta(days=90))

df_insiderbuys0 = (df_insidertrades
                  .where((df_insidertrades['type'] == 'Buy') & (df_insidertrades['date'] >= datefilter))
                  .dropna(axis=0, how='all').groupby(['ticker_id', 'exchange_id']).sum())


df_insiderbuys = (df_master.set_index(['ticker_id', 'exchange_id'])
                  .join(df_insiderbuys0, how='inner').reset_index()
                  .groupby(['company', 'sector', 'industry']).mean().round(1)
                  .sort_values(by='value', ascending=False)
                 )[['openprice', 'yield', 'quantity', 'value']]

print('Total of {:,.0f} records meet this criterium.'.format(len(df_insiderbuys)))

In [ ]:
with open('doc/df_insiderbuys.csv', 'w') as file:
    file.write(df_insiderbuys.to_csv())

[return to top of this section](#value),
[return to the top](#top)
<a id="mergerules"></a>
### Merging DataFrames

In [ ]:
df_master.columns.values

In [ ]:
df_rules = (df_master[df_master['security_type'] == 'Stock']
            #.merge(df_rule0_Rev, on=['ticker_id', 'exchange_id'])    # CAGR > 7% for past 5 yrs - Revenue
            #.merge(df_rule0_OpeInc, on=['ticker_id', 'exchange_id']) # CAGR > 7% for past 5 yrs - Ope. Income
            #.merge(df_rule0_OpeCF, on=['ticker_id', 'exchange_id'])  # CAGR > 7% for past 5 yrs - Ope. Cash Flow
            #.merge(df_rule0_FreeCF, on=['ticker_id', 'exchange_id']) # CAGR > 7% for past 5 yrs - Free Cash Flow
            .merge(df_rule1_5, on=['ticker_id', 'exchange_id'])      # No earnings deficit for past 5 yrs
            .merge(df_rule2, on=['ticker_id', 'exchange_id'])        # Uniterrupted Dividends for past 7 yrs
            .merge(df_rule3, on=['ticker_id', 'exchange_id'])        # P/E Ratio of 10 or less for past 7 yrs
            .merge(df_rule4_Rev, on=['ticker_id', 'exchange_id'])    # Growth for the past year - Revenue
            .merge(df_rule4_OpeInc, on=['ticker_id', 'exchange_id']) # Growth for the past year - Ope. Income
            .merge(df_rule4_NetInc, on=['ticker_id', 'exchange_id']) # Growth for the past year - Net Income
            .merge(df_rule5, on=['ticker_id', 'exchange_id'])        # Current Ratio > 1.2
            .merge(df_rule6, on=['ticker_id', 'exchange_id'])        # Debt/Equity < 1.0
            .merge(df_rule7, on=['ticker_id', 'exchange_id'])        # Return on Equity > 10%
            .merge(df_rule8, on=['ticker_id', 'exchange_id'])        # P/B < 1.0
            #.merge(df_insiderbuys0, on=['ticker_id', 'exchange_id'])  # Insider buys in the past 3 months
            #.merge(df_vals[['ticker_id', 'exchange_id', 'PB_TTM', 'PS_TTM', 'PC_TTM']],
            #       on=['ticker_id', 'exchange_id'])

            #.groupby(['company', 'exchange_sym', 'ticker', 'sector', 'industry']).mean().round(1)
           )[['company', 'exchange_sym', 'ticker', 'sector', 'industry',
              'lastprice', 'yield', '_52wk_hi', '_52wk_lo',
              'PE_TTM_x', 'PB_TTM', 'PS_TTM', 'PC_TTM', 'current_ratio', 'debt2equity', 'return_on_equity',
              #'CAGR_Rev', 'CAGR_OpeInc', 'CAGR_OpeCF', 'CAGR_FreeCF',
              'Rev_Growth_Y9', 'OpeInc_Growth_Y9', 'NetInc_Growth_Y9'#, 'value'
              ]]

total_companies = df_master[df_master['security_type'] == 'Stock']#.groupby('company').count()
msg = 'A total of {:,.0f} stocks meet these criteria out of {:,.0f} (as of {})'
print(msg.format(len(df_rules), len(total_companies), DT.date.today()))

In [ ]:
df_rules

In [ ]:
with open('doc/df_rules.csv', 'w') as file:
    file.write(df_rules.to_csv())

<a id="additional"></a>
[return to the top](#top)

## Additional sample / test code

In [ ]:
import requests

In [ ]:
url = 'http://performance.morningstar.com/perform/Performance/stock/exportStockPrice.action?t={}:{}&pd=1yr&freq=d&pg=0&culture=en-US'
req = requests.get(url.format('xnas', 'aaoi'))